## Reactivation strength

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.gridspec as gridspec
from scipy.ndimage import gaussian_filter
from ccg import correlograms
from mathutil import getICA_Assembly
from callfunc import processData
import subjects
from plotUtil import Fig


In [ ]:
# region
plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(16, 1, figure=fig)
fig.subplots_adjust(hspace=0.3)
sessions = subjects.Sd().ratSday3
for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    maze = sess.epochs.maze
    post = sess.epochs.post
    activation = sess.replay.assemblyICA.detect(template=maze, match=post)

    for i in range(16):
        ax = fig.add_subplot(gs[i])
        ax.plot(activation[i])

# sess.brainstates.detect()
# endregion

# violations = sess.spikes.stability.violations

## Reactivation strength vs delta amplitude

In [ ]:
# region
plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(1, 1, figure=fig)
fig.subplots_adjust(hspace=0.3)

for sub, sess in enumerate(sessions):
    sess.trange = np.array([])

    maze = sess.epochs.maze
    post = sess.epochs.post
    activation, t = sess.replay.assemblyICA.detect(template=maze, match=post)

    swa = sess.swa.time


# endregion



## similarity of ICA vectors detected during MAZE and last hour of sleep deprivation 

In [ ]:
# region
plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(3, 3, figure=fig)
fig.subplots_adjust(hspace=0.3)

for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    spikes = sess.spikes.times
    pre = sess.epochs.pre
    maze = sess.epochs.maze
    post = sess.epochs.post
    bins = [
        [maze[0], maze[1]],
        [post[0], post[0] + 3600],
        [post[0] + 4 * 3600, post[0] + 5 * 3600],
        # [post[0] + 5 * 3600, post[0] + 6 * 3600],
    ]
    sess.spikes.stability.firingRate(bins=bins)
    stability = sess.spikes.stability.info
    stable_pyr = np.where((stability.q < 4) & (stability.stable == 1))[0]

    pyr = [spikes[cell_id] for cell_id in stable_pyr]
    maze_bin = np.arange(bins[0][0], bins[0][1], 0.25)
    spkcnt_maze = np.asarray([np.histogram(cell, bins=maze_bin)[0] for cell in pyr])
    sd_bin = np.arange(bins[1][0], bins[1][1], 0.25)
    spkcnt_zt5 = np.asarray([np.histogram(cell, bins=sd_bin)[0] for cell in pyr])
    rec_bin = np.arange(bins[2][0], bins[2][1], 0.25)
    spkcnt_rec = np.asarray([np.histogram(cell, bins=rec_bin)[0] for cell in pyr])

    v_maze = sess.replay.assemblyICA.getAssemblies(spkcnt_maze)
    v_sd = sess.replay.assemblyICA.getAssemblies(spkcnt_zt5)
    v_rec = sess.replay.assemblyICA.getAssemblies(spkcnt_rec)

    # v_maze = np.where(v_maze > 0, 1, 0)
    # v_sd = np.where(v_sd > 0, 1, 0)
    # v_rec = np.where(v_rec > 0, 1, 0)

    ica_similarity_maze_rec = np.abs(np.matmul(v_maze.T, v_sd))
    ica_similarity_sd_rec = np.abs(np.matmul(v_maze.T, v_rec))

    ax = fig.add_subplot(gs[sub, 0])
    ax.bar([1, 2], [np.mean(ica_similarity_maze_rec), np.mean(ica_similarity_sd_rec)])

    ax = fig.add_subplot(gs[sub, 1])
    ax.imshow(np.abs(ica_similarity_maze_rec), vmax=1, vmin=0)

    ax = fig.add_subplot(gs[sub, 2])
    ax.imshow(np.abs(ica_similarity_sd_rec), vmax=1, vmin=0)


# endregion


## Activation strength during sleep deprivation

In [ ]:
# region

plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(1, 3, figure=fig)
fig.subplots_adjust(hspace=0.3, wspace=0.2)
for sub, sess in enumerate(sessions[:3]):
    sess.trange = np.array([])
    maze = sess.epochs.maze
    post = sess.epochs.post
    spks = sess.spikes.times
    sd_period = [maze[0], post[0] + 5 * 3600]
    intervals = sess.utils.getinterval(period=sd_period, nwindows=5)
    sess.spikes.stability.firingRate(bins=intervals)
    stability = sess.spikes.stability.info
    stable_pyr = np.where((stability.q < 4) & (stability.stable == 1))[0]

    pyr = [spks[cell_id] for cell_id in stable_pyr]
    print(len(pyr))
    # sd_bin = np.arange(post[0], post[0] + 5 * 3600, 0.25)
    # spkcnt_sd = np.array([np.histogram(cell, bins=sd_bin)[0] for cell in pyr])

    activation, match_bin = sess.replay.assemblyICA.getActivation(
        template=maze, match=[sd_period[0], sd_period[1]], spks=pyr
    )

    ax = plt.subplot(gs[sub])
    sess.replay.assemblyICA.plotActivation(ax=ax)
    ax.set_title(sess.sessinfo.session.sessionName)

# endregion


## SWR vs reactivation strength during sleep deprivation

In [ ]:
# region

plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(1, 4, figure=fig)
fig.subplots_adjust(hspace=0.3, wspace=0.2)
for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    maze = sess.epochs.maze
    post = sess.epochs.post
    spks = sess.spikes.times
    stability_period = [maze[0], post[0] + 5 * 3600]
    intervals = sess.utils.getinterval(period=stability_period, nwindows=5)
    sess.spikes.stability.firingRate(bins=intervals)
    stability = sess.spikes.stability.info
    stable_pyr = np.where((stability.q < 4) & (stability.stable == 1))[0]

    pyr = [spks[cell_id] for cell_id in stable_pyr]
    print(len(pyr))

    sd_period = [post[0], post[0] + 5 * 3600]
    activation, match_bin = sess.replay.assemblyICA.getActivation(
        template=maze, match=[sd_period[0], sd_period[1]], spks=pyr
    )

    # sess.replay.assemblyICA.plotActivation(ax=plt.subplot(gs[sub]))

    sum_activation = np.sum(stats.zscore(activation, axis=1), axis=0)

    ripple = sess.ripple.time
    ripple_zt1 = ripple[
        (ripple[:, 0] > sd_period[0]) & (ripple[:, 0] < sd_period[0] + 3600), :
    ]
    ripple_zt5 = ripple[
        (ripple[:, 0] > sd_period[0] + 4 * 3600)
        & (ripple[:, 0] < sd_period[0] + 5 * 3600),
        :,
    ]

    mean_react_zt1 = stats.binned_statistic(
        match_bin[1:] - 0.125, sum_activation, bins=np.concatenate(ripple_zt1)
    )[0][::2]
    mean_react_zt5 = stats.binned_statistic(
        match_bin[1:] - 0.125, sum_activation, bins=np.concatenate(ripple_zt5)
    )[0][::2]

    ax = plt.subplot(gs[sub])
    ax.bar([1, 2], [np.nanmean(mean_react_zt1), np.nanmean(mean_react_zt5)])
# endregion


## delta vs reactivation strength during sleep deprivation

In [ ]:
# region

plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(1, 4, figure=fig)
fig.subplots_adjust(hspace=0.3, wspace=0.2)
for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    maze = sess.epochs.maze
    post = sess.epochs.post
    spks = sess.spikes.times
    stability_period = [maze[0], post[0] + 5 * 3600]
    intervals = sess.utils.getinterval(period=stability_period, nwindows=5)
    sess.spikes.stability.firingRate(bins=intervals)
    stability = sess.spikes.stability.info
    stable_pyr = np.where((stability.q < 4) & (stability.stable == 1))[0]

    pyr = [spks[cell_id] for cell_id in stable_pyr]
    print(len(pyr))

    sd_period = [post[0], post[0] + 5 * 3600]
    activation, match_bin = sess.replay.assemblyICA.getActivation(
        template=maze, match=[sd_period[0], sd_period[1]], spks=pyr
    )

    # sess.replay.assemblyICA.plotActivation(ax=plt.subplot(gs[sub]))

    sum_activation = np.sum(stats.zscore(activation, axis=1), axis=0)

    ripple = sess.swa.time
    ripple_zt1 = ripple[
        (ripple[:, 0] > sd_period[0]) & (ripple[:, 0] < sd_period[0] + 3600), :
    ]
    ripple_zt5 = ripple[
        (ripple[:, 0] > sd_period[0] + 4 * 3600)
        & (ripple[:, 0] < sd_period[0] + 5 * 3600),
        :,
    ]

    mean_react_zt1 = stats.binned_statistic(
        match_bin[1:] - 0.125, sum_activation, bins=np.concatenate(ripple_zt1)
    )[0][::2]
    mean_react_zt5 = stats.binned_statistic(
        match_bin[1:] - 0.125, sum_activation, bins=np.concatenate(ripple_zt5)
    )[0][::2]

    ax = plt.subplot(gs[sub])
    ax.bar([1, 2], [np.nanmean(mean_react_zt1), np.nanmean(mean_react_zt5)])
# endregion



## Firing rate before and after swr comparison between sleep and swr

In [ ]:
# region

plt.clf()
fig = plt.figure(1, figsize=(10, 15))
gs = gridspec.GridSpec(1, 4, figure=fig)
fig.subplots_adjust(hspace=0.3, wspace=0.2)
for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    maze = sess.epochs.maze
    post = sess.epochs.post
    spks = sess.spikes.times
    stability_period = [post[0], post[0] + 5 * 3600]
    intervals = sess.utils.getinterval(period=stability_period, nwindows=5)
    sess.spikes.stability.firingRate(bins=intervals)
    stability = sess.spikes.stability.info
    stable_pyr = np.where((stability.q < 4) & (stability.stable == 1))[0]

    pyr = [spks[cell_id] for cell_id in stable_pyr]
    print(len(pyr))
    spks = np.concatenate(pyr)

    sd_period = [post[0], post[0] + 5 * 3600]

    ripple = sess.ripple.time
    ripple_sd = ripple[(ripple[:, 0] > sd_period[0]) & (ripple[:, 0] < sd_period[1]), :]

    instfiring = sess.spikes.instfiring

    instfire_rpl = np.zeros(1999)
    for rpl in ripple_sd:
        instfire_rpl += instfiring.loc[
            (instfiring.time > rpl[0] - 1) & (instfiring.time < rpl[0] + 1), "frate"
        ].to_numpy()[:1999]

    ax = plt.subplot(gs[sub])
    ax.plot(instfire_rpl / len(ripple_sd))

# endregion


## Directed reward vs sprinkled reward reactivation comparison

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(4, 5), size=(12, 6))
sessions = subjects.Of().ratNday4
for sub, sess in enumerate(sessions):
    maze = sess.epochs.maze
    pre = sess.epochs.pre
    post = sess.epochs.post
    sprinkle = sess.epochs.sprinkle
    pre_sprinkle = [maze[0], sprinkle[0]]

    x = sess.position.x
    y = sess.position.y
    t = sess.position.t

    maze_indx = np.where((t > maze[0]) & (t < maze[1]))[0]
    maze_x = x[maze_indx]
    maze_y = y[maze_indx]
    xbins = np.linspace(np.min(maze_x), np.max(maze_x), 40)
    ybins = np.linspace(np.min(maze_y), np.max(maze_y), 40)
    xgrid, ygrid = np.meshgrid(xbins, ybins)

    sess.replay.assemblyICA.getAssemblies(period=pre_sprinkle)
    activation_pre_sprinkle, t_pre_sprinkle = sess.replay.assemblyICA.getActivation(
        period=pre_sprinkle
    )
    activation_sprinkle, t_sprinkle = sess.replay.assemblyICA.getActivation(
        period=sprinkle
    )
    # sess.replay.assemblyICA.plotActivation()

    act_presprinkle_x = np.interp(t_pre_sprinkle, t, x)
    act_presprinkle_y = np.interp(t_pre_sprinkle, t, y)

    act_sprinkle_x = np.interp(t_sprinkle, t, x)
    act_sprinkle_y = np.interp(t_sprinkle, t, y)

    act_pre_sprinkle_zscore = stats.zscore(activation_pre_sprinkle, axis=1)
    act_pre_sprinkle_zscore = np.where(act_pre_sprinkle_zscore > 2, 1, 0)

    act_sprinkle_zscore = stats.zscore(activation_sprinkle, axis=1)
    act_sprinkle_zscore = np.where(act_sprinkle_zscore > 2, 1, 0)

    for i, (ensemble_pre, ensemble_sprinkle) in enumerate(
        zip(act_pre_sprinkle_zscore, act_sprinkle_zscore)
    ):
        gs_ = figure.subplot2grid(gs[i], grid=(1, 2), wspace=0.1)
        ax = plt.subplot(gs_[0])
        indices = np.where(ensemble_pre == 1)[0]
        hist_ = np.histogram2d(
            act_presprinkle_x[indices], act_presprinkle_y[indices], bins=[xbins, ybins]
        )[0]
        hist_ = gaussian_filter(hist_, sigma=1)
        # plt.plot(act_presprinkle_x[indices], act_presprinkle_y[indices], "k.")
        ax.pcolormesh(xgrid, ygrid, hist_.T, cmap="hot")
        ax.axis("off")
        # ax.plot(maze_x, maze_y, alpha=0.3)
        indices = np.where(ensemble_sprinkle == 1)[0]
        hist_ = np.histogram2d(
            act_sprinkle_x[indices], act_sprinkle_y[indices], bins=[xbins, ybins]
        )[0]
        ax = plt.subplot(gs_[1])
        # plt.plot(act_sprinkle_x[indices], act_sprinkle_y[indices], "k.")
        hist_ = gaussian_filter(hist_.T, sigma=1)
        ax.pcolormesh(xgrid, ygrid, hist_, cmap="hot")
        ax.axis("off")

    # figure = Fig()
    # fig, gs = figure.draw(num=2, grid=(18, 1))
    # fig = plt.figure(num=2)
    fig, ax = plt.subplots(nrows=18, ncols=2, num=2, sharex="col", sharey="row")

    for i, (ensemble_pre, ensemble_sprinkle) in enumerate(
        zip(activation_pre_sprinkle, activation_sprinkle)
    ):
        # gs_ = figure.subplot2grid(gs[i], grid=(1, 2))
        # ax = plt.subplot(gs_[0])
        ax[i, 0].plot(t_pre_sprinkle[1:], ensemble_pre)

        # ax = plt.subplot(gs_[1], sharey=ax)
        ax[i, 1].plot(t_sprinkle[1:], ensemble_sprinkle)


# endregion


## Test maze epochs for sprinkles vs pre-sprinkls

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(4, 5), size=(12, 6))
sessions = subjects.Of().ratNday4
for sub, sess in enumerate(sessions):
    maze = sess.epochs.maze
    pre = sess.epochs.pre
    post = sess.epochs.post
    sprinkle = sess.epochs.sprinkle
    pre_sprinkle = [maze[0], sprinkle[0]]

    x = sess.position.x
    y = sess.position.y
    z = sess.position.z
    t = sess.position.t

    indices = np.where((t > pre_sprinkle[0]) & (t < pre_sprinkle[1]))[0]
    testx = x[indices]
    testy = y[indices]
    plt.plot(testx, testy)

# endregion



## Assembly activity during MAZE2 experience

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(2, 2))
sessions = subjects.Sd().ratSday3 + subjects.Nsd().ratSday2
for sub, sess in enumerate(sessions):
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2

    sess.replay.assemblyICA.getAssemblies(period=maze1)
    activation_maze2, t_maze2 = sess.replay.assemblyICA.getActivation(period=maze2)
    # sess.replay.assemblyICA.plotActivation()

    hist_ = np.histogram(activation_maze2, bins=np.arange(0, 100, 2), density=True)[0]
    # cdf = np.cumsum(hist_)
    plt.plot(hist_)
# endregion



## Reactivation participating cells locatioin on maze for each ensemble

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(10, 10))
sessions = subjects.Of().ratNday4
for sub, sess in enumerate(sessions):
    maze = sess.epochs.maze
    sprinkle = sess.epochs.sprinkle
    sess.replay.assemblyICA.getAssemblies(period=maze)
    vectors = sess.replay.assemblyICA.vectors
    activation_maze2, t_maze2 = sess.replay.assemblyICA.getActivation(period=maze)

    spikes = sess.replay.assemblyICA.spikes
    sess.placefield.pf2d.compute(period=maze, spikes=spikes, gridbin=5, smooth=2)
    maps = sess.placefield.pf2d.maps
    xgrid = sess.placefield.pf2d.xgrid
    ygrid = sess.placefield.pf2d.ygrid
    x = 5

    for i, assembly in enumerate(vectors.T):
        ax = plt.subplot(gs[i, 0])
        sigweight = 1.5 * np.std(assembly)
        cellid = np.where(assembly > sigweight)[0]
        print(len(cellid))

        for cell_num, cell in enumerate(cellid):
            pf_loc = np.unravel_index(np.argmax(maps[cell]), shape=maps[cell].shape)
            x, y = xgrid[pf_loc[0]], ygrid[pf_loc[1]]
            ax.plot(x, y, "*")

            axmap = plt.subplot(gs[i, cell_num + 1])
            axmap.pcolormesh(maps[cell], cmap="Spectral_r")

        ax.set_xlim([np.min(xgrid), np.max(xgrid)])
        ax.set_ylim([np.min(ygrid), np.max(ygrid)])

    # pf = sess.placefield.pf2d.compute(period=maze)


# endregion



## Activation strength along theta oscillation

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(2, 1))
sessions = subjects.Sd().ratSday3
for sub, sess in enumerate(sessions):
    maze = sess.epochs.maze1
    lfp = sess.recinfo.geteeg(chans=113, timeRange=maze)
    t = np.linspace(maze[0], maze[1], len(lfp))
    sess.replay.assemblyICA.getAssemblies(period=maze)
    activation_maze, t_maze2 = sess.replay.assemblyICA.getActivation(period=maze)

    activation_maze = stats.zscore(activation_maze, axis=1)

    ax = plt.subplot(gs[0])
    ax.plot(t, lfp, "k")

    axact = plt.subplot(gs[1], sharex=ax)
    for i, assembly in enumerate(activation_maze):
        indices = np.where(assembly > 1.5)[0]
        t_locs = t_maze2[indices]
        axact.plot(t_locs, i * np.ones(len(t_locs)), "|")


# endregion


## Two novel maze assembly comaprison

In [ ]:
# region
figure = Fig()
fig, gs = figure.draw(num=1, grid=(2, 2))
sessions = subjects.Tn().ratSday5
for sub, sess in enumerate(sessions):
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2

    assembly_maze1 = sess.replay.assemblyICA.getAssemblies(maze1)
    assembly_maze2 = sess.replay.assemblyICA.getAssemblies(maze2)

    assemblies = np.hstack((assembly_maze1, assembly_maze2))
    assembly_corr = np.corrcoef(assemblies.T)
    np.fill_diagonal(assembly_corr, 0)

    ax = plt.subplot(gs[0])
    im = ax.imshow(assembly_corr, cmap="jet")
    ax.set_xlabel("Assembly index")
    ax.set_ylabel("Assembly index")
    plt.colorbar(im)
# endregion


## Cells selectivity for each environment (two novel maze)


In [ ]:
import numpy as np
import scipy.signal as sg
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig
from scipy import stats

In [ ]:
figure = Fig()
fig, gs = figure.draw(num=1, grid=(3, 2))
sessions = subjects.Tn().ratSday5
for sub, sess in enumerate(sessions):
    pre=sess.epochs.pre
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2
    post2 = sess.epochs.post2

    spks = sess.spikes.pyr
    cell_ids = sess.spikes.pyrid
    nspikes_maze1 = np.asarray([np.histogram(cell, bins=maze1)[0] for cell in spks])
    nspikes_maze2 = np.asarray([np.histogram(cell, bins=maze2)[0] for cell in spks])

    #---- remove low firing cells --------
    good_cells_maze1 = cell_ids[np.where(nspikes_maze1>10)[0]]
    good_cells_maze2 = cell_ids[np.where(nspikes_maze2>10)[0]]
    good_cells_ids = np.intersect1d(good_cells_maze1,good_cells_maze2)

    #----- firing rate -------
    maze_dur = np.diff(maze1)+np.diff(maze2)
    good_cells_spks = sess.spikes.get_cells(good_cells_ids)
    frate = [np.histogram(cell,bins=maze1+maze2)[0][::2].sum()/maze_dur for cell in good_cells_spks]
    frate = np.asarray(frate)
    frate_color = frate-np.min(frate)/(np.max(frate)-np.min(frate))
    assembly_maze1 = sess.replay.assemblyICA.getAssemblies(good_cells_ids,maze1)
    activation_maze1 = sess.replay.assemblyICA.getActivation(maze1)[0]
    assembly_maze2 = sess.replay.assemblyICA.getAssemblies(good_cells_ids,maze2)
    activation_maze2 = sess.replay.assemblyICA.getActivation(maze2)[0]

    mean_weight_maze1 = stats.zscore(np.mean(assembly_maze1,axis=1))
    mean_weight_maze2 = stats.zscore(np.mean(assembly_maze2,axis=1))

    # assemblies = np.hstack((assembly_maze1, assembly_maze2))
    # assembly_corr = np.corrcoef(assemblies.T)
    # np.fill_diagonal(assembly_corr, 0)

    ax = plt.subplot(gs[0])
    ax.spines.left.set_position('zero')
    ax.spines.bottom.set_position('zero')
    ax.scatter(mean_weight_maze1,mean_weight_maze2,s=frate*30,c=frate_color,cmap='coolwarm')
    # ax.set_xlabel("maze1")
    # ax.set_ylabel("maze2")
    # ax.set_aspect('equal', adjustable='datalim')
    # ax.set_xlim([-0.1,0.1])
    # ax.set_xlim([-0.1,0.1])
    # ax.set_xticks([-0.05,0,0.05])
    # ax.set_yticks([-0.05,0,0.05])
    # plt.colorbar(im)

    ax = plt.subplot(gs[1])
    frate_maze1 = sess.spikes.firing_rate(good_cells_ids,maze1)
    frate_maze2 = sess.spikes.firing_rate(good_cells_ids,maze2)
    ax.scatter(frate_maze1,frate_maze2,c=frate_color,cmap='coolwarm')
    ax.set_xscale('log')
    ax.set_yscale('log')

    ax = plt.subplot(gs[2])
    corr,time = sess.spikes.corr.across_time_window(good_cells_ids,period=[pre[0],post2[1]])
    np.fill_diagonal(corr,val=0)
    time = time/3600
    im = ax.pcolormesh(time,time,corr,shading = 'gouraud',cmap='jet')
    ax2 = plt.subplot(gs[3])
    plt.colorbar(im,ax=ax2)



## Two-novel activation strength of maze1 plotted on maze2 and show ripple periods
* Also compare mean activation in non-ripple periods vs ripple-periods 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import subjects
from plotUtil import Fig

In [9]:
%matplotlib qt
figure = Fig()
fig,gs = figure.draw(num=1,grid=(2,1))
sessions = subjects.Tn().ratSday5
for sub,sess in enumerate(sessions):
    rpls = sess.ripple.events
    pyrid = sess.spikes.pyrid
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2

    assembly_maze1 = sess.replay.assemblyICA.getAssemblies(pyrid,maze1)
    activation_maze2,t_maze2 = sess.replay.assemblyICA.getActivation(period=maze2)

    mean_activation = np.mean(activation_maze2,axis=0)

    ax = plt.subplot(gs[0])
    ax.plot(t_maze2[:-1],mean_activation,'k')
    sess.ripple.plot_ripples(maze2,ax)
    ax.set_xlabel('Maze2 time (seconds)')
    ax.set_ylabel('Mean activation strength')
    ax.legend(['','ripples'])